# Classification Schoolwork
### milestone2
#### author：Hsu An      2016011434
#### last update: 3 Dec 2018

update:
1. 使用LightGBM在未进行调参的情况下进行测试，acc为0.9270758122743682

In [111]:
import scipy.io as scio
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import lightgbm as lgb   

from sklearn.metrics import accuracy_score

## 1.数据预处理

In [112]:
# 读入数据
dataset = np.array([])
file_num = [2,3,5,6,8,10,11,12,14]
for i in file_num:
    dataFile = 'dataset/data'+str(i)+'_train.mat'
    data = scio.loadmat(dataFile)
    data = np.array(data['data'+str(i)+'_train'])
    label = np.full((len(data)),i)
    data = np.c_[data,label]
    dataset = np.append(dataset,data)

# 将一维数据转化为行列形式
dataset = dataset.reshape(-1,201)
dataset

array([[3.688e+03, 4.381e+03, 4.594e+03, ..., 1.034e+03, 1.010e+03,
        2.000e+00],
       [2.579e+03, 4.137e+03, 4.412e+03, ..., 1.043e+03, 1.005e+03,
        2.000e+00],
       [2.581e+03, 4.264e+03, 4.328e+03, ..., 1.022e+03, 1.009e+03,
        2.000e+00],
       ...,
       [2.725e+03, 3.982e+03, 4.099e+03, ..., 1.004e+03, 1.008e+03,
        1.400e+01],
       [3.320e+03, 3.862e+03, 4.015e+03, ..., 1.004e+03, 1.000e+03,
        1.400e+01],
       [3.154e+03, 3.986e+03, 4.099e+03, ..., 9.960e+02, 1.003e+03,
        1.400e+01]])

In [113]:
dataset = pd.DataFrame(dataset)
dataset.rename(columns={ dataset.columns[200]: "y" }, inplace=True) # rename不是传递对象

In [114]:
dataset

,0,1,2,3,4,5,6,7,8,9,...,191,192,193,194,195,196,197,198,199,y
0,3688.0,4381.0,4594.0,4415.0,4944.0,5287.0,5470.0,5363.0,5364.0,5123.0,...,1114.0,1123.0,1123.0,1103.0,1071.0,1078.0,1064.0,1034.0,1010.0,2.0
1,2579.0,4137.0,4412.0,4343.0,4825.0,5228.0,5359.0,5184.0,5185.0,5020.0,...,1111.0,1113.0,1119.0,1089.0,1075.0,1081.0,1051.0,1043.0,1005.0,2.0
2,2581.0,4264.0,4328.0,4334.0,4758.0,5116.0,5129.0,5015.0,4958.0,4788.0,...,1110.0,1095.0,1103.0,1086.0,1067.0,1073.0,1054.0,1022.0,1009.0,2.0
3,2747.0,4377.0,4412.0,4196.0,4635.0,4969.0,4990.0,4920.0,4863.0,4620.0,...,1099.0,1096.0,1100.0,1078.0,1066.0,1085.0,1042.0,1029.0,1023.0,2.0
4,3178.0,4013.0,4324.0,4258.0,4530.0,4959.0,4909.0,4801.0,4725.0,4614.0,...,1089.0,1086.0,1092.0,1073.0,1066.0,1065.0,1040.0,1034.0,1015.0,2.0
5,2749.0,4257.0,4324.0,4129.0,4635.0,4866.0,4962.0,4807.0,4767.0,4568.0,...,1084.0,1087.0,1098.0,1080.0,1058.0,1067.0,1045.0,1021.0,1014.0,2.0
6,2579.0,4137.0,4328.0,4343.0,4535.0,4915.0,5043.0,4846.0,4791.0,4507.0,...,1101.0,1095.0,1100.0,1078.0,1065.0,1063.0,1041.0,1024.0,1019.0,2.0
7,2584.0,4253.0,4421.0,4334.0,4697.0,4915.0,4985.0,4841.0,4719.0,4540.0,...,1091.0,1089.0,1100.0,1081.0,1066.0,1054.0,1053.0,1029.0,1014.0,2.0
8,3693.0,4005.0,4510.0,4201.0,4582.0,4920.0,4866.0,4791.0,4719.0,4540.0,...,1088.0,1092.0,1096.0,1082.0,1053.0,1053.0,1033.0,1025.0,1019.0,2.0
9,3688.0,4009.0,4501.0,4343.0,4578.0,4969.0,4942.0,4712.0,4725.0,4528.0,...,1088.0,1092.0,1095.0,1078.0,1056.0,1061.0,1039.0,1020.0,1014.0,2.0


In [115]:
dataset_label = dataset['y']
dataset = dataset.drop(['y'], axis=1)

In [117]:
# # PCA降维
# from sklearn.decomposition import PCA
# spca = PCA()
# dataset = spca.fit_transform(dataset)

# # LLE降维
# from sklearn.manifold import LocallyLinearEmbedding
# LLE = LocallyLinearEmbedding()
# dataset = LLE.fit_transform(dataset)

# LDA降维
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
LDA = LinearDiscriminantAnalysis()
dataset = LDA.fit_transform(dataset,dataset_label)

In [118]:
# 划分单独测试集
X_train, X_test, y_train, y_test = train_test_split(dataset,dataset_label,test_size=0.2,random_state=1) 

## 2. 模型选择

In [119]:
# # 0.28231046931407944
# from sklearn import svm
# clf = svm.NuSVC(nu=0.2)

# # 0.859927797833935
# from sklearn.ensemble import RandomForestClassifier
# clf = RandomForestClassifier()

# # 0.9025270758122743
# from sklearn.ensemble import GradientBoostingClassifier
# clf = GradientBoostingClassifier()

# # 0.27292418772563176
# from sklearn.ensemble import AdaBoostClassifier
# clf = AdaBoostClassifier()

# # 0.41732851985559566
# from sklearn.neural_network import MLPClassifier
# clf = MLPClassifier()

# # 0.8743682310469314
# import xgboost as xgb
# from xgboost.sklearn import XGBClassifier
# clf = xgb.XGBClassifier() 

# from catboost import CatBoostClassifier
# clf = CatBoostClassifier(loss_function='MultiClass') 

# from sklearn.svm import SVC
# clf = SVC(kernel='rbf',gamma=0.125)

clf = lgb.LGBMClassifier(application='multiclass',objective='multiclass')

In [120]:
# 学习模型
clf.fit(X_train,y_train)
y_pre = clf.predict(X_test)

# 统计结果  
print(accuracy_score(y_test,y_pre))

0.9068592057761733


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [91]:
print(y_pre)

[14. 11. 11. ...  2. 11. 11.]


## 3. 结果预测

In [97]:
test = scio.loadmat('./dataset/data_test_final.mat')
test = np.array(test['data_test_final'])

# PCA降维
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
LDA = LinearDiscriminantAnalysis()
test = LDA.fit_transform(test)

test
test_pre = clf.predict(test)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
